In [1]:
kdprov = '76'

In [2]:
# Parameters
kdprov = "12"


In [3]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt

from dbfread import DBF
from glob import glob
from tqdm import tqdm
import os 

import geopandas as gpd
import rasterio
import rasterio.features as features
from rasterio.plot import show
from shapely.geometry import Polygon

from osgeo import ogr
from osgeo import osr
from osgeo import gdal

import itertools
import rasterio
from shapely.geometry import box
import geopandas as gpd
from rasterio.enums import MergeAlg

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
from osgeo import gdal, ogr, osr

In [5]:
data_kab=gpd.read_file('data/input/Master_Data/kab_20201_byindo.gpkg').query('kdprov==@kdprov')

In [6]:
from rasterio.mask import mask

In [7]:
import json
src  = rasterio.open('data/input/GHSL Data/2020/prov/2020_'+kdprov+'.tif')
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]
for i in data_kab.kdkab.unique():
    df=data_kab.query('kdkab==@i').to_crs('ESRI:54009')
    
    coords = getFeatures(df)
    clipped_array, clipped_transform = mask(dataset=src, shapes=coords, crop=True)

    df = df.to_crs(src.crs)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": clipped_array.shape[1],
                 "width": clipped_array.shape[2],
                 "transform": clipped_transform})
    out_tif= 'data/input/GHSL Data/2020/prov/2020_'+kdprov+'_'+i+'.tif'
    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(clipped_array)

In [8]:
def ghsl_kab_convert(dt):
    with rasterio.open(dt) as dataset:
        data = dataset.read(1)

        t = dataset.transform

        move_x = t[0]
        move_y = t[4]

        height = dataset.height
        width = dataset.width 

        polygons = []
        indices = list(itertools.product(range(width), range(height)))
        data_list = []
    
        for x,y in indices:
            x_min, y_max = t * (x,y)
            x_max = x_min + move_x
            y_min = y_max + move_y
            polygons.append(box(x_min, y_min, x_max, y_max))
            data_list.append(data[y,x])
    gdf = gpd.GeoDataFrame(data=data_list, geometry=polygons, columns=['value'])
    kdkab=dt.split('_')[2].replace('.tif','')
    gdf.query('(value>=0)and(value<=10000)').to_file("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+kdkab+"_nonzero.shp")

In [9]:
import itertools
import rasterio
from shapely.geometry import box
import geopandas as gpd
from glob import glob as glob
from tqdm.contrib.concurrent import process_map

data_rasterio_kab=glob('data/input/GHSL Data/2020/prov/2020_'+kdprov+'_*.tif')
process_map(ghsl_kab_convert,data_rasterio_kab)   

  0%|          | 0/34 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [10]:
# import gpkg desa & filter

vector = gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov==@kdprov').to_crs("ESRI:54009")
raster = rasterio.open(r"/vsigs/bps-gcp-bucket/Degree of Urbanisation/GHSL data/2020/prov/2020_"+kdprov+".tif")
geom = [shapes for shapes in vector.geometry]
# vector['iddesa'] = vector.iddesa.astype(np.int32)
vector['id'] = range(0,len(vector))

In [11]:
vector['geometry']=vector.geometry.buffer(0)

In [12]:
# create tuples of geometry, value pairs, where value is the attribute value you want to burn
geom_value = ((geom,value) for geom, value in zip(vector.geometry, vector['id']))
# Rasterize vector using the shape and transform of the raster
#rasterized = features.rasterize(geom_value,
#                                out_shape = raster.shape,
#                                fill = -5,
#                                out = None,
#                                transform = raster.transform,
#                                all_touched = False,
#                                default_value = 1,
#                                dtype = np.int32)

rasterized = features.rasterize(geom_value,
                                out_shape = raster.shape,
                                transform = raster.transform,
                                all_touched = True,
                                fill = -5,   # background value
                                default_value = 1,
                                dtype = np.float64)

# Plot raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized, ax = ax)
#plt.gca().invert_yaxis()

<AxesSubplot:>

In [13]:
# rasterization polygon desa

with rasterio.open(
        "data/temp/rasterized_vector.tif", "w",
        driver = "GTiff",
        transform = raster.transform,
        dtype = rasterio.float64,
        count = 1,
        width = raster.width,
        height = raster.height) as dst:
    dst.write(rasterized, indexes = 1)

# polygonize polygon desa

raster_src = gdal.Open(r'data/temp/rasterized_vector.tif')
band = raster_src.GetRasterBand(1) 

drv = ogr.GetDriverByName('ESRI Shapefile')
outfile = drv.CreateDataSource(r'data/temp/polygonizedRaster.shp') 
outlayer = outfile.CreateLayer('polygonized raster', srs = None )
newField = ogr.FieldDefn('DN', ogr.OFTReal)
outlayer.CreateField(newField)

gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None

raster = None
raster_src = None

desa = gpd.read_file(r'data/temp/polygonizedRaster.shp')
desa = desa.merge(vector[['id','iddesa']], 
                                 left_on = "DN", right_on = "id", how="left").dissolve(by="iddesa").reset_index()
desa.crs='ESRI:54009'
desa.to_file("data/input/desa_rasterize_"+kdprov+".gpkg", driver="GPKG")

In [14]:
# import ruta

data_ruta=pd.read_csv('data/input/Master_Data/pop_desa_rev.csv',sep=',')
data_ruta.iddesa=data_ruta.iddesa.astype(str)
#data_ruta['iddesa'] = [x[0:10] for x in data_ruta.KODE_SLS]
data_ruta['idprov'] = [x[0:2] for x in data_ruta.iddesa]

data_ruta = data_ruta.loc[data_ruta.idprov.isin([kdprov])]

# join penduduk & polygon desa

data_ruta_desa = pd.DataFrame(data_ruta.groupby("iddesa").pop2020.sum()).reset_index()
desa = desa.merge(data_ruta_desa, how="left").fillna(0)
desa.set_crs('ESRI:54009').to_file("data/output/0_raster_data_desa/desa_pend_rasterize_"+kdprov+".gpkg", driver="GPKG")

In [15]:
print("Ada di Peta. tidak di Hasil SP : "+str(desa[~desa.iddesa.isin(data_ruta.iddesa)].shape[0]))
print("Ada di Hasil SP. tidak di Peta : "+str(data_ruta[~data_ruta.iddesa.isin(desa.iddesa)].shape[0]))

Ada di Peta. tidak di Hasil SP : 2
Ada di Hasil SP. tidak di Peta : 1


In [16]:
data_ruta[~data_ruta.iddesa.isin(desa.iddesa)]

Unnamed: 0      iddesa  pop2020 idprov
9385        9385  1212090022      199     12